# Project 3 Dashboard Data Cleaning

### Import Dependancies

In [9]:
# Import dependencies
import pandas as pd
from flask import Flask, jsonify
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime as dt
from datetime import timedelta
import re

### Read in raw data

In [2]:
# Read in the full dataset
df = pd.read_csv('Crime_Data_from_2020_to_Present.csv')


### Explore data set

In [3]:
# visualize
df

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752906,231510379,05/29/2023 12:00:00 AM,05/25/2023 12:00:00 AM,1100,15,N Hollywood,1548,2,662,"BUNCO, GRAND THEFT",...,IC,Invest Cont,662.0,NaN,NaN,NaN,5300 DENNY AV,NaN,34.1667,-118.3643
752907,231604807,01/27/2023 12:00:00 AM,01/26/2023 12:00:00 AM,1800,16,Foothill,1663,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,12500 BRANFORD ST,NaN,34.2466,-118.4054
752908,231606525,03/22/2023 12:00:00 AM,03/22/2023 12:00:00 AM,1000,16,Foothill,1602,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230.0,NaN,NaN,NaN,12800 FILMORE ST,NaN,34.2790,-118.4116
752909,231210064,04/12/2023 12:00:00 AM,04/12/2023 12:00:00 AM,1630,12,77th Street,1239,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230.0,NaN,NaN,NaN,6100 S VERMONT AV,NaN,33.9841,-118.2915


In [4]:
# visualize the columns
df.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

### Clean data 

In [5]:
# drop unnessesary columns
df = df.drop(['DR_NO','Date Rptd', 'Rpt Dist No', 'Part 1-2', 'Mocodes','Vict Descent', 'Premis Cd','Weapon Used Cd', 'Status','Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4'], axis=1)

In [6]:
# visualize
df

,DATE OCC,TIME OCC,AREA,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Premis Desc,Weapon Desc,Status Desc,Crm Cd 1,LOCATION,Cross Street,LAT,LON
0,01/08/2020 12:00:00 AM,2230,3,Southwest,624,BATTERY - SIMPLE ASSAULT,36,F,SINGLE FAMILY DWELLING,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Adult Other,624.0,1100 W 39TH PL,NaN,34.0141,-118.2978
1,01/01/2020 12:00:00 AM,330,1,Central,624,BATTERY - SIMPLE ASSAULT,25,M,SIDEWALK,UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,624.0,700 S HILL ST,NaN,34.0459,-118.2545
2,02/13/2020 12:00:00 AM,1200,1,Central,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,POLICE FACILITY,NaN,Adult Arrest,845.0,200 E 6TH ST,NaN,34.0448,-118.2474
3,01/01/2020 12:00:00 AM,1730,15,N Hollywood,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,745.0,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,415,19,Mission,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",31,X,BEAUTY SUPPLY STORE,NaN,Invest Cont,740.0,14400 TITUS ST,NaN,34.2198,-118.4468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752906,05/25/2023 12:00:00 AM,1100,15,N Hollywood,662,"BUNCO, GRAND THEFT",25,M,OTHER PREMISE,NaN,Invest Cont,662.0,5300 DENNY AV,NaN,34.1667,-118.3643
752907,01/26/2023 12:00:00 AM,1800,16,Foothill,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",23,M,"VEHICLE, PASSENGER/TRUCK",NaN,Invest Cont,740.0,12500 BRANFORD ST,NaN,34.2466,-118.4054
752908,03/22/2023 12:00:00 AM,1000,16,Foothill,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",25,F,SIDEWALK,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,230.0,12800 FILMORE ST,NaN,34.2790,-118.4116
752909,04/12/2023 12:00:00 AM,1630,12,77th Street,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",29,M,LAUNDROMAT,UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,230.0,6100 S VERMONT AV,NaN,33.9841,-118.2915


In [7]:
# visualize negative age data
negative = df.loc[df['Vict Age'] == -2]
negative

,DATE OCC,TIME OCC,AREA,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Premis Desc,Weapon Desc,Status Desc,Crm Cd 1,LOCATION,Cross Street,LAT,LON
18061,12/15/2020 12:00:00 AM,2359,11,Northeast,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,-2,X,SINGLE FAMILY DWELLING,NaN,Invest Cont,845.0,3300 SAN FERNANDO RD,NaN,34.1180,-118.2489
20847,05/20/2020 12:00:00 AM,1600,17,Devonshire,354,THEFT OF IDENTITY,-2,X,OTHER BUSINESS,NaN,Invest Cont,354.0,19500 BUSINESS CENTER DR,NaN,34.2313,-118.5564
198921,05/18/2020 12:00:00 AM,1130,21,Topanga,354,THEFT OF IDENTITY,-2,M,OTHER BUSINESS,NaN,Invest Cont,354.0,23100 VENTURA BL,NaN,34.1637,-118.6289
198927,03/04/2020 12:00:00 AM,800,18,Southeast,946,OTHER MISCELLANEOUS CRIME,-2,X,POLICE FACILITY,NaN,Invest Cont,946.0,100 W 108TH ST,NaN,33.9401,-118.2761
199012,04/22/2020 12:00:00 AM,1200,12,77th Street,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",-2,M,SINGLE FAMILY DWELLING,NaN,Invest Cont,341.0,7600 DALTON AV,NaN,33.9709,-118.3035
199075,06/02/2020 12:00:00 AM,1,4,Hollenbeck,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,-2,X,POLICE FACILITY,NaN,Invest Cont,845.0,2100 E 1ST ST,NaN,34.0449,-118.2138
199177,05/31/2020 12:00:00 AM,430,6,Hollywood,310,BURGLARY,-2,X,OTHER BUSINESS,UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,310.0,1700 ORCHID AV,NaN,34.1030,-118.3402
199207,06/01/2020 12:00:00 AM,1200,14,Pacific,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,-2,M,SINGLE FAMILY DWELLING,VERBAL THREAT,Invest Cont,930.0,3500 GRAND VIEW BL,NaN,34.0118,-118.4365
199258,06/02/2020 12:00:00 AM,1,17,Devonshire,354,THEFT OF IDENTITY,-2,M,SINGLE FAMILY DWELLING,NaN,Invest Cont,354.0,16800 SAN FERNANDO MISSION BL,NaN,34.2720,-118.4978
205404,01/01/2021 12:00:00 AM,1,18,Southeast,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,-2,X,POLICE FACILITY,NaN,Invest Cont,845.0,100 E 119TH ST,NaN,33.9247,-118.2739


In [8]:
# remove negative age data
cleanedDf = df.drop(df[df['Vict Age'] < 1].index)
cleanedDf

,DATE OCC,TIME OCC,AREA,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Premis Desc,Weapon Desc,Status Desc,Crm Cd 1,LOCATION,Cross Street,LAT,LON
0,01/08/2020 12:00:00 AM,2230,3,Southwest,624,BATTERY - SIMPLE ASSAULT,36,F,SINGLE FAMILY DWELLING,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Adult Other,624.0,1100 W 39TH PL,NaN,34.0141,-118.2978
1,01/01/2020 12:00:00 AM,330,1,Central,624,BATTERY - SIMPLE ASSAULT,25,M,SIDEWALK,UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,624.0,700 S HILL ST,NaN,34.0459,-118.2545
3,01/01/2020 12:00:00 AM,1730,15,N Hollywood,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,745.0,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,01/01/2020 12:00:00 AM,415,19,Mission,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",31,X,BEAUTY SUPPLY STORE,NaN,Invest Cont,740.0,14400 TITUS ST,NaN,34.2198,-118.4468
5,01/01/2020 12:00:00 AM,30,1,Central,121,"RAPE, FORCIBLE",25,F,NIGHT CLUB (OPEN EVENINGS ONLY),UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,121.0,700 S BROADWAY,NaN,34.0452,-118.2534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752906,05/25/2023 12:00:00 AM,1100,15,N Hollywood,662,"BUNCO, GRAND THEFT",25,M,OTHER PREMISE,NaN,Invest Cont,662.0,5300 DENNY AV,NaN,34.1667,-118.3643
752907,01/26/2023 12:00:00 AM,1800,16,Foothill,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",23,M,"VEHICLE, PASSENGER/TRUCK",NaN,Invest Cont,740.0,12500 BRANFORD ST,NaN,34.2466,-118.4054
752908,03/22/2023 12:00:00 AM,1000,16,Foothill,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",25,F,SIDEWALK,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,230.0,12800 FILMORE ST,NaN,34.2790,-118.4116
752909,04/12/2023 12:00:00 AM,1630,12,77th Street,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",29,M,LAUNDROMAT,UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,230.0,6100 S VERMONT AV,NaN,33.9841,-118.2915


### Export cleaned full data set to CSV

In [ ]:
cleanedDf.to_csv("DataCleaned.csv", index=False)

### Export sample data set to CSV

In [9]:
# grab first 1000 rows
df = cleanedDf.head(1000)

In [23]:
# export sample of data for git upload purposes
df.to_csv("DataSample.csv", index=False)